In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import sys
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Circle
import seaborn as sns; 
from IPython.display import HTML

In [ ]:
#config parser
import configparser

sys.path.insert(0, '../Common/')
from AISDataManager import AISDataManager
import Constants as c
import HMUtils as hMUtil
import TimeUtils as timeUtils
import GeoCompute as gC

#MyConfig.INI stores all the run time constants
config = configparser.ConfigParser()
config.read('../MyConfig.INI')

from joblib import Parallel, delayed
import multiprocessing
aISDM = AISDataManager()

In [ ]:
lonMin = (float)(config['TRAJ_PRED_LSTM_3']['LON_MIN'])
lonMax = (float)(config['TRAJ_PRED_LSTM_3']['LON_MAX'])

latMin = (float)(config['TRAJ_PRED_LSTM_3']['LAT_MIN'])
latMax = (float)(config['TRAJ_PRED_LSTM_3']['LAT_MAX'])

print(lonMin,latMin)
print(lonMax,latMax)

increStep = (float)(config['TRAJ_PRED_LSTM_3']['INCR_STEP'])
incrRes = (int)(config['TRAJ_PRED_LSTM_3']['INCR_RES'])

sourceDir = config['TRAJ_PRED_LSTM_3']['SOURCE_DIR']
trainTrajNum = (int)(config['TRAJ_PRED_LSTM_3']['TRAIN_DATA'])
testEndTrajNum = (int)(config['TRAJ_PRED_LSTM_3']['TEST_END'])
dataDir = config['TRAJ_PRED_LSTM_3']['DATA_DIR']
print(sourceDir)
print(trainTrajNum)
print(testEndTrajNum)
print(dataDir)

In [ ]:
heatMapGrid = hMUtil.generate_grid(lonMin, lonMax, latMin, latMax, increStep, incrRes)
boundaryArray = heatMapGrid[2]
horizontalAxis = heatMapGrid[0]
verticalAxis = heatMapGrid[1]
totalStates = horizontalAxis.shape[0] * verticalAxis.shape[0]

In [ ]:
#first read the data frame
#convert LON and LAT into cooresponding 
# def get_index_from_lon_lat(lat,lon):
def get_index_from_lon_lat(latLonRow):
    retVal = -1
    lat = latLonRow['LAT']
    lon = latLonRow['LON']
    for boundary in boundaryArray: 
        if(lon >= boundary[0]) and (lon < boundary[1]) \
            and (lat >= boundary[2]) and (lat < boundary[3]):
            retVal = boundary[4]
            break 
    return retVal

#will convert dataframe into sequence of numbers
#which can be used to make data for LSTM
def convert_traj_df_to_state_sequence(num):
    #read the dataframe
    sorceFile = sourceDir + str(num) + '.csv'
    sourceDF,_ = aISDM.load_data_from_csv(sorceFile)
    #conver every LON and LAT to sequence of numbers
    ret = sourceDF.apply(get_index_from_lon_lat,axis=1)
    return ret.to_numpy()

def get_traj_lon_lat_data(num):
    #read the dataframe
    sorceFile = sourceDir + str(num) + '.csv'
    sourceDF,_ = aISDM.load_data_from_csv(sorceFile)
    return sourceDF.loc[:,['LON','LAT']].to_numpy()
    
# convert_traj_df_to_state_sequence(0)
get_traj_lon_lat_data(0)

In [ ]:
#make list of all such trajectories
#number goes from 0 to 80000
#that much is going to be our training data
trajSeqList = []

#FIXME get number from Config file
for trajNum in range(0,trainTrajNum):
    trajSeqList.append(get_traj_lon_lat_data(trajNum))

In [ ]:
print(trajSeqList[0])

In [ ]:
#takes numpy array and returns x and y for
#time series prediction
def convert_seq_to_x_y(seq):
    #first column
    #-2 is is to take care of boundary condition
    #since we are considering 2 time stamps for the input data
    firstCol = seq[:-2].copy()
    firstCol = np.reshape(firstCol,(firstCol.shape[0],1))
#     print(firstCol)
#     print(firstCol.shape)
    #second column shifted by 1 time instances
    secCol = seq[1:-1].copy()
    secCol = np.reshape(secCol,(secCol.shape[0],1))
#     print(secCol)
#     print(secCol.shape)
    #output is shifted by two time instances
    outputLabel = seq[2:].copy()
    outputLabel = np.reshape(outputLabel,(outputLabel.shape[0],1))
#     print(outputLabel)
#     print(outputLabel.shape)
    xData = np.hstack((firstCol,secCol))
#     print(xData.shape)
    return xData, outputLabel

def convert_seq_to_x_y_lon_lat(seq, prevTimeStamp = 2):
    #first column
    #-2 is is to take care of boundary condition
    #since we are considering 2 time stamps for the input data
    firstTwoCol = seq[:-3,:].copy()
    secTwoCol = seq[1:-2].copy()
    thirdTwoCol = seq[2:-1].copy()
    #output is shifted by two time instances
    outputLabel = seq[3:].copy()
    xData = np.hstack((firstTwoCol,secTwoCol,thirdTwoCol))
    return xData, outputLabel
    
#convert_seq_to_x_y_lon_lat(trajSeqList[0])

In [ ]:
#now iterate throgh trajSeqList 
#and keep on stacking them vertically
#to make giant input and output matrix
xData = np.zeros((0,6))
yData = np.zeros((0,2))
print(xData.shape)
print(yData.shape)
for trajNum in range(len(trajSeqList)):
    if((trajSeqList[trajNum].shape[0]) > 3):
        xTemp,yTemp = convert_seq_to_x_y_lon_lat(trajSeqList[trajNum])
        xData = np.vstack((xData,xTemp.copy()))
        yData = np.vstack((yData,yTemp.copy()))
        
print(xData)
print(yData)

In [ ]:
print(xData.shape)
print(yData.shape)

In [ ]:
xToStore = dataDir + "XData.npy"
yToStore = dataDir + "YData.npy"
np.save(xToStore, xData)
np.save(yToStore, yData)

In [ ]:
xToStore = dataDir + "XData.npy"
yToStore = dataDir + "YData.npy"

xData = np.load(xToStore)
yData = np.load(yToStore)

In [ ]:
print(xData.shape)
print(yData.shape)

In [ ]:
#normalise the data
xLonLatData_0 = (xData[:,0] - lonMin)/(lonMax - lonMin)
xLonLatData_0 = np.reshape(xLonLatData_0,(xLonLatData_0.shape[0],1))
xLonLatData_1 = (xData[:,1] - latMin)/(latMax - latMin)
xLonLatData_1 = np.reshape(xLonLatData_1,(xLonLatData_1.shape[0],1))
xLonLatData_2 = (xData[:,2] - lonMin)/(lonMax - lonMin)
xLonLatData_2 = np.reshape(xLonLatData_2,(xLonLatData_2.shape[0],1))
xLonLatData_3 = (xData[:,3] - latMin)/(latMax - latMin)
xLonLatData_3 = np.reshape(xLonLatData_3,(xLonLatData_3.shape[0],1))

xLonLatData_4 = (xData[:,4] - lonMin)/(lonMax - lonMin)
xLonLatData_4 = np.reshape(xLonLatData_4,(xLonLatData_4.shape[0],1))
xLonLatData_5 = (xData[:,5] - latMin)/(latMax - latMin)
xLonLatData_5 = np.reshape(xLonLatData_5,(xLonLatData_5.shape[0],1))

xLonLatDataNorm = np.hstack((xLonLatData_0,xLonLatData_1,xLonLatData_2,xLonLatData_3, xLonLatData_4, xLonLatData_5))
xLonLatDataNorm = np.reshape(xLonLatDataNorm,(xLonLatDataNorm.shape[0], 3, 2))

In [ ]:
print(xLonLatDataNorm.shape)

In [ ]:
#normalise the output data as well
yLonLatData_0 = (yData[:,0] - lonMin)/(lonMax - lonMin)
yLonLatData_0 = np.reshape(yLonLatData_0,(yLonLatData_0.shape[0],1))
yLonLatData_1 = (yData[:,1] - latMin)/(latMax - latMin)
yLonLatData_1 = np.reshape(yLonLatData_1,(yLonLatData_1.shape[0],1))
yLonLatDataNorm = np.hstack((yLonLatData_0,yLonLatData_1))

In [ ]:
print(yLonLatDataNorm.shape)

In [ ]:
print(xLonLatDataNorm[0,:,:])
print(yLonLatDataNorm[0,:])

In [ ]:
from keras import Sequential
from keras.layers import Dense, LSTM
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences= True, input_shape=(3,2)))
model.add(LSTM(units=50))
model.add(Dense(150, activation='relu'))
model.add(Dense(units=2, activation='linear'))
# model = Sequential()
# model.add(LSTM(units=250, return_sequences= True, input_shape=(2,2)))
# model.add(LSTM(units=250))
# model.add(Dense(150, activation='relu'))
# model.add(Dense(units=2, activation='linear'))
# visible = Input(shape=(2,2))
# hidden1 = LSTM(250, return_sequences= True)(visible)
# hidden2 = LSTM(250)(hidden1)
# lonLatDense = Dense(150, activation='relu')(hidden2)
# lonLatOp = Dense(2, activation='linear')(lonLatDense)
# modelFunc = Model(inputs=visible, outputs=lonLatOp)
# modelFunc.summary()

In [ ]:
# visible = Input(shape=(2,2))
# hidden1 = LSTM(250, return_sequences= True)(visible)
# hidden2 = LSTM(250)(hidden1)
# lonLatDense = Dense(150, activation='relu')(hidden2)
# lonLatOp = Dense(2, activation='linear')(lonLatDense)
# modelFunc = Model(inputs=visible, outputs=lonLatOp)
# modelFunc.summary()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(xLonLatDataNorm, yLonLatDataNorm, epochs=500, batch_size = 512 , verbose = 2)

In [ ]:
modelDir = dataDir + "Model_500_MSE_3.h5"

In [ ]:
model.save(modelDir)

In [ ]:
from keras.models import load_model
model = load_model(modelDir)

In [ ]:
#will take four arguments
#[[lonPrev, latPrev, lonCurr, latCurr]] shape is 1x4 or nx4
#can be vectorised also
def normalize_lon_lat(arr):
    #subtract the minimum 
    #and divide by range
    ret0 = (arr[:,0] - lonMin)/(lonMax - lonMin)
    ret0 = np.reshape(ret0, (ret0.shape[0],1))
    ret1 = (arr[:,1] - latMin)/(latMax - latMin)
    ret1 = np.reshape(ret1, (ret1.shape[0],1))
    ret = np.hstack((ret0, ret1))
    return ret

In [ ]:
#takes 2 time stamps of LON and LAT
#normalises them and 
#and makes the prediction
#de normalize the output
#and return the values
#prevTraj numpy array of 1x2
#currTraj numpy array of 1x2
def compute_30_min_pred(prevPrevTraj, prevTraj, currTraj):
    """
    Compute prediction for 30 minutes.
    
    takes LON and LAT of two time stamps
    normalises them
    and makes prediction
    and returns denormalised LON and LAT values
    """
    if(prevPrevTraj.shape != (1,2)):
        raise Exception('Shape of prevPrevTraj should be 1x2')
        
    if(prevTraj.shape != (1,2)):
        raise Exception('Shape of prevTraj should be 1x2')
    
    if(currTraj.shape != (1,2)):
        raise Exception('Shape of currTraj should be 1x2')
        
    trajX = np.vstack((prevPrevTraj,prevTraj,currTraj))
    trajXNorm = normalize_lon_lat(trajX)
    
    trajXNorm = np.reshape(trajXNorm,(1,3,2))
    predLatLon = model.predict(trajXNorm)
    
    predLon = predLatLon[0,0]
    predLat = predLatLon[0,1]
    
    #after prediction de normalise it
    predLonScaled = (predLon * (lonMax - lonMin)) + lonMin
    predLatScaled = (predLat * (latMax - latMin)) + latMin
    return predLonScaled, predLatScaled

In [ ]:
def compute_n_30_min_pred(prevPrevTraj, prevTraj, currTraj, n = 1):
    """
    Compute prediction for n*30 minutes.
    
    calls compute_30_min_pred
    according to value of n
    default value of n = 1
    """
    #temp vaariables to store the previous trajectory 
    firstLoc = prevPrevTraj.copy()
    secLoc = prevTraj.copy()
    thirdLoc = currTraj.copy()
    
    #return value will be numpy array of nx2
    ret = np.zeros((0,2))
    
    for i in range(n):
        #make prediction using 30 min pred
        predLon, predLat = compute_30_min_pred(firstLoc, secLoc, thirdLoc)
        #append them in ret
        ret = np.vstack((ret,np.array([[predLon,predLat]])))
        
        #update firstLoc and secLoc 
        #for next  iteration
        
        firstLoc = secLoc.copy()
        secLoc = thirdLoc.copy()
        thirdLoc = np.reshape(ret[-1,:].copy(),(1,2))
    return ret

In [ ]:
def get_error_for_traj(num):
    """
    Compute error for the entire trajectory.
    
    Returns list of error for predictions of upto 
    4 hours i.e. 240 minutes.
    
    Parameters:
        num (int): number of vessel trajectory 
                   whose prediction errors to be computed.
    Returns:
        errorVal (list of floats): list of errors in KM for each
                                   30 min predictions.
    """    
    errorVal = []
    vesselTraj = get_traj_lon_lat_data(num)
    
    if(vesselTraj.shape[0] < 4):
        return errorVal
    
    predVesselTraj = compute_n_30_min_pred(np.reshape(vesselTraj[0,:], (1,2)), np.reshape(vesselTraj[1,:], (1,2)), np.reshape(vesselTraj[2,:], (1,2)), n = 8)
    predRange = vesselTraj.shape[0] - 3

    #FIXME 8 can come from some variable too
    #for 8 consecutive predictions
    if(predRange > 8):
        predRange = 8

    #use trajectories which has atleast one
    #value to predict
    if(predRange >= 0):
        for pred in range(predRange):    
            trueLon, trueLat = vesselTraj[(3 + pred),0], vesselTraj[(3 + pred),1]
            predLon, predLat = predVesselTraj[pred,0], predVesselTraj[pred,1]
    #         print(trueLon, trueLat, '-', predLon, predLat)
            errorVal.append(gC.compute_distance(trueLon, trueLat, predLon, predLat))
    return errorVal

In [ ]:
help(get_error_for_traj)

In [ ]:
#store errors for all training trajectories
trainDataWholeErrors = []
for traj in range(trainTrajNum):
    trainDataWholeErrors.append(get_error_for_traj(traj))

In [ ]:
#segregate those list of errors into
#list of 30 minErr, 60 minErr, ...
trainWholeErr_30 = []
trainWholeErr_60 = []
trainWholeErr_90 = []
trainWholeErr_120 = []
trainWholeErr_150 = []
trainWholeErr_180 = []
trainWholeErr_210 = []
trainWholeErr_240 = []

trainWholeErr_n_30 = [trainWholeErr_30 \
, trainWholeErr_60 \
, trainWholeErr_90 \
, trainWholeErr_120 \
, trainWholeErr_150 \
, trainWholeErr_180 \
, trainWholeErr_210 \
, trainWholeErr_240 \
]

for trajErr in trainDataWholeErrors:
    #take the list
    for n_30 in range(8):
        if(len(trajErr) > n_30):
            trainWholeErr_n_30[n_30].append(trajErr[n_30])

In [ ]:
def format_func(value, tick_number):
    tempTick = (value*30) + 30
    ret = "%d"%(tempTick)
    return ret

In [ ]:
trainWholeErr_30NP = np.array(trainWholeErr_30)
trainWholeErr_60NP = np.array(trainWholeErr_60)
trainWholeErr_90NP = np.array(trainWholeErr_90)
trainWholeErr_120NP = np.array(trainWholeErr_120)
trainWholeErr_150NP = np.array(trainWholeErr_150)
trainWholeErr_180NP = np.array(trainWholeErr_180)
trainWholeErr_210NP = np.array(trainWholeErr_210)
trainWholeErr_240NP = np.array(trainWholeErr_240)

In [ ]:
trainWholeErrMean = [np.mean(trainWholeErr_30NP) \
            ,np.mean(trainWholeErr_60NP) \
            ,np.mean(trainWholeErr_90NP) \
            ,np.mean(trainWholeErr_120NP) \
            ,np.mean(trainWholeErr_150NP) \
            ,np.mean(trainWholeErr_180NP) \
            ,np.mean(trainWholeErr_210NP) \
            ,np.mean(trainWholeErr_240NP) \
           ]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title("Average Error Value On Training")
ax.set_ylabel("Distance in KM")
ax.set_xlabel("Time in Minutes")
ax.plot(trainWholeErrMean,label = "LSTM")
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.legend()

In [ ]:
#store errors for all testing trajectories
testDataWholeErrors = []
for traj in range(trainTrajNum, testEndTrajNum):
    testDataWholeErrors.append(get_error_for_traj(traj))

In [ ]:
#segregate those list of errors into
#list of 30 minErr, 60 minErr, ...
testWholeErr_30 = []
testWholeErr_60 = []
testWholeErr_90 = []
testWholeErr_120 = []
testWholeErr_150 = []
testWholeErr_180 = []
testWholeErr_210 = []
testWholeErr_240 = []

testWholeErr_n_30 = [testWholeErr_30 \
, testWholeErr_60 \
, testWholeErr_90 \
, testWholeErr_120 \
, testWholeErr_150 \
, testWholeErr_180 \
, testWholeErr_210 \
, testWholeErr_240 \
]

for trajErr in testDataWholeErrors:
    #take the list
    for n_30 in range(8):
        if(len(trajErr) > n_30):
            testWholeErr_n_30[n_30].append(trajErr[n_30])

In [ ]:
testWholeErr_30NP = np.array(testWholeErr_30)
testWholeErr_60NP = np.array(testWholeErr_60)
testWholeErr_90NP = np.array(testWholeErr_90)
testWholeErr_120NP = np.array(testWholeErr_120)
testWholeErr_150NP = np.array(testWholeErr_150)
testWholeErr_180NP = np.array(testWholeErr_180)
testWholeErr_210NP = np.array(testWholeErr_210)
testWholeErr_240NP = np.array(testWholeErr_240)

In [ ]:
testWholeErrMean = [np.mean(testWholeErr_30NP) \
            ,np.mean(testWholeErr_60NP) \
            ,np.mean(testWholeErr_90NP) \
            ,np.mean(testWholeErr_120NP) \
            ,np.mean(testWholeErr_150NP) \
            ,np.mean(testWholeErr_180NP) \
            ,np.mean(testWholeErr_210NP) \
            ,np.mean(testWholeErr_240NP) \
           ]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title("Average Error Value On Testing")
ax.set_ylabel("Distance in KM")
ax.set_xlabel("Time in Minutes")
ax.plot(testWholeErrMean,label = "LSTM")
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.legend()

In [ ]:
np.random.seed(0)
rand1000TrajTrain = np.random.randint(0,trainTrajNum,size = 1000).tolist()

In [ ]:
#store errors for all training trajectories
trainR1000Err = []
for traj in rand1000TrajTrain:
    trainR1000Err.append(get_error_for_traj(traj))

In [ ]:
#segregate those list of errors into
#list of 30 minErr, 60 minErr, ...
trainR1000Err_30 = []
trainR1000Err_60 = []
trainR1000Err_90 = []
trainR1000Err_120 = []
trainR1000Err_150 = []
trainR1000Err_180 = []
trainR1000Err_210 = []
trainR1000Err_240 = []

trainR1000Err_n_30 = [trainR1000Err_30 \
, trainR1000Err_60 \
, trainR1000Err_90 \
, trainR1000Err_120 \
, trainR1000Err_150 \
, trainR1000Err_180 \
, trainR1000Err_210 \
, trainR1000Err_240 \
]

for trajErr in trainR1000Err:
    #take the list
    for n_30 in range(8):
        if(len(trajErr) > n_30):
            trainR1000Err_n_30[n_30].append(trajErr[n_30])

In [ ]:
trainR1000Err_30NP = np.array(trainR1000Err_30)
trainR1000Err_60NP = np.array(trainR1000Err_60)
trainR1000Err_90NP = np.array(trainR1000Err_90)
trainR1000Err_120NP = np.array(trainR1000Err_120)
trainR1000Err_150NP = np.array(trainR1000Err_150)
trainR1000Err_180NP = np.array(trainR1000Err_180)
trainR1000Err_210NP = np.array(trainR1000Err_210)
trainR1000Err_240NP = np.array(trainR1000Err_240)

In [ ]:
trainR1000ErrMean = [np.mean(trainR1000Err_30NP) \
            ,np.mean(trainR1000Err_60NP) \
            ,np.mean(trainR1000Err_90NP) \
            ,np.mean(trainR1000Err_120NP) \
            ,np.mean(trainR1000Err_150NP) \
            ,np.mean(trainR1000Err_180NP) \
            ,np.mean(trainR1000Err_210NP) \
            ,np.mean(trainR1000Err_240NP) \
           ]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title("Average Error for 1000 Training Trajectories")
ax.set_ylabel("Distance in KM")
ax.set_xlabel("Time in Minutes")
ax.plot(trainR1000ErrMean,label = "LSTM")
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.legend()

In [ ]:
np.random.seed(1)
rand1000TrajTest = np.random.randint(trainTrajNum,testEndTrajNum,size = 1000).tolist()

In [ ]:
#store errors for all testing trajectories
testR1000Err = []
for traj in rand1000TrajTest:
    testR1000Err.append(get_error_for_traj(traj))

In [ ]:
#segregate those list of errors into
#list of 30 minErr, 60 minErr, ...
testR1000Err_30 = []
testR1000Err_60 = []
testR1000Err_90 = []
testR1000Err_120 = []
testR1000Err_150 = []
testR1000Err_180 = []
testR1000Err_210 = []
testR1000Err_240 = []

testR1000Err_n_30 = [testR1000Err_30 \
, testR1000Err_60 \
, testR1000Err_90 \
, testR1000Err_120 \
, testR1000Err_150 \
, testR1000Err_180 \
, testR1000Err_210 \
, testR1000Err_240 \
]

for trajErr in testR1000Err:
    #take the list
    for n_30 in range(8):
        if(len(trajErr) > n_30):
            testR1000Err_n_30[n_30].append(trajErr[n_30])

In [ ]:
testR1000Err_30NP = np.array(testR1000Err_30)
testR1000Err_60NP = np.array(testR1000Err_60)
testR1000Err_90NP = np.array(testR1000Err_90)
testR1000Err_120NP = np.array(testR1000Err_120)
testR1000Err_150NP = np.array(testR1000Err_150)
testR1000Err_180NP = np.array(testR1000Err_180)
testR1000Err_210NP = np.array(testR1000Err_210)
testR1000Err_240NP = np.array(testR1000Err_240)

In [ ]:
testR1000ErrMean = [np.mean(testR1000Err_30NP) \
            ,np.mean(testR1000Err_60NP) \
            ,np.mean(testR1000Err_90NP) \
            ,np.mean(testR1000Err_120NP) \
            ,np.mean(testR1000Err_150NP) \
            ,np.mean(testR1000Err_180NP) \
            ,np.mean(testR1000Err_210NP) \
            ,np.mean(testR1000Err_240NP) \
           ]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.set_title("Average Error for 1000 Testing Trajectories")
ax.set_ylabel("Distance in KM")
ax.set_xlabel("Time in Minutes")
ax.plot(testR1000ErrMean,label = "LSTM")
ax.xaxis.set_major_formatter(plt.FuncFormatter(format_func))
ax.legend()

In [ ]:
dataToStore = dataDir + "trainWholeErr_30NP.npy"
np.save(dataToStore, trainWholeErr_30NP)
dataToStore = dataDir + "trainWholeErr_60NP.npy"
np.save(dataToStore, trainWholeErr_60NP)
dataToStore = dataDir + "trainWholeErr_90NP.npy"
np.save(dataToStore, trainWholeErr_90NP)
dataToStore = dataDir + "trainWholeErr_120NP.npy"
np.save(dataToStore, trainWholeErr_120NP)
dataToStore = dataDir + "trainWholeErr_150NP.npy"
np.save(dataToStore, trainWholeErr_150NP)
dataToStore = dataDir + "trainWholeErr_180NP.npy"
np.save(dataToStore, trainWholeErr_180NP)
dataToStore = dataDir + "trainWholeErr_210NP.npy"
np.save(dataToStore, trainWholeErr_210NP)
dataToStore = dataDir + "trainWholeErr_240NP.npy"
np.save(dataToStore, trainWholeErr_240NP)

In [ ]:
dataToStore = dataDir + "testWholeErr_30NP.npy"
np.save(dataToStore, testWholeErr_30NP)
dataToStore = dataDir + "testWholeErr_60NP.npy"
np.save(dataToStore, testWholeErr_60NP)
dataToStore = dataDir + "testWholeErr_90NP.npy"
np.save(dataToStore, testWholeErr_90NP)
dataToStore = dataDir + "testWholeErr_120NP.npy"
np.save(dataToStore, testWholeErr_120NP)
dataToStore = dataDir + "testWholeErr_150NP.npy"
np.save(dataToStore, testWholeErr_150NP)
dataToStore = dataDir + "testWholeErr_180NP.npy"
np.save(dataToStore, testWholeErr_180NP)
dataToStore = dataDir + "testWholeErr_210NP.npy"
np.save(dataToStore, testWholeErr_210NP)
dataToStore = dataDir + "testWholeErr_240NP.npy"
np.save(dataToStore, testWholeErr_240NP)

In [ ]:
dataToStore = dataDir + "trainR1000Err_30NP.npy"
np.save(dataToStore, trainR1000Err_30NP)
dataToStore = dataDir + "trainR1000Err_60NP.npy"
np.save(dataToStore, trainR1000Err_60NP)
dataToStore = dataDir + "trainR1000Err_90NP.npy"
np.save(dataToStore, trainR1000Err_90NP)
dataToStore = dataDir + "trainR1000Err_120NP.npy"
np.save(dataToStore, trainR1000Err_120NP)
dataToStore = dataDir + "trainR1000Err_150NP.npy"
np.save(dataToStore, trainR1000Err_150NP)
dataToStore = dataDir + "trainR1000Err_180NP.npy"
np.save(dataToStore, trainR1000Err_180NP)
dataToStore = dataDir + "trainR1000Err_210NP.npy"
np.save(dataToStore, trainR1000Err_210NP)
dataToStore = dataDir + "trainR1000Err_240NP.npy"
np.save(dataToStore, trainR1000Err_240NP)

In [ ]:
dataToStore = dataDir + "testR1000Err_30NP.npy"
np.save(dataToStore, testR1000Err_30NP)
dataToStore = dataDir + "testR1000Err_60NP.npy"
np.save(dataToStore, testR1000Err_60NP)
dataToStore = dataDir + "testR1000Err_90NP.npy"
np.save(dataToStore, testR1000Err_90NP)
dataToStore = dataDir + "testR1000Err_120NP.npy"
np.save(dataToStore, testR1000Err_120NP)
dataToStore = dataDir + "testR1000Err_150NP.npy"
np.save(dataToStore, testR1000Err_150NP)
dataToStore = dataDir + "testR1000Err_180NP.npy"
np.save(dataToStore, testR1000Err_180NP)
dataToStore = dataDir + "testR1000Err_210NP.npy"
np.save(dataToStore, testR1000Err_210NP)
dataToStore = dataDir + "testR1000Err_240NP.npy"
np.save(dataToStore, testR1000Err_240NP)